calculate phylogenic alpha diversity for rarified biom data, with greengenes97 tree

In [9]:
import redbiom.summarize
import redbiom.search
import redbiom.fetch
import qiime2
from qiime2.plugins import feature_table, diversity
import pandas as pd
from biom import load_table

In [5]:
btable = load_table('./rar1000_greengenes/feature-table.biom')

In [6]:
btable

19134 x 20304 <class 'biom.table.Table'> with 2519259 nonzero entries (0% dense)

In [11]:
table_ar = qiime2.Artifact.import_data('FeatureTable[Frequency]', btable)

## Skbio Grenegenes tree load

fix tree issue - https://forum.qiime2.org/t/greengenes-tree-download-with-branch-lengths/3329/5

In [13]:
import skbio

t = skbio.TreeNode.read('97_otus.tree')

for n in t.traverse():
    if n.length is None:
        n.length = 0.0


# import qiime2
ar = qiime2.Artifact.import_data('Phylogeny[Rooted]', t)
# ar.save('97_otus.qza')

calculate alpha diversity with qiime library

In [14]:
alpha_div = diversity.actions.alpha_phylogenetic(table=table_ar, phylogeny=ar, metric='faith_pd')

In [15]:
alpha_div.alpha_diversity.export_data('alpha_div')

directly calculate alpha_div with skbio instead (I dont like biomTable structure, its confusing)

In [16]:
counts = btable.matrix_data.toarray().astype(int).T

In [17]:
sample_ids = btable.ids(axis='sample')
feature_ids = btable.ids(axis='observation')

In [18]:
result = skbio.diversity.alpha_diversity(metric='faith_pd',
                                         counts=counts,
                                         ids=sample_ids,
                                         otu_ids=feature_ids,
                                         tree=t)


In [19]:
print(len(result))
result[:5]

20304


10317.000023681.3099     14.05699
10317.000028593.3022     13.11953
10317.000041303.26484    10.39727
10317.000022953.3018     13.64948
10317.000015004.2917      5.67176
dtype: float64